In [37]:
'''
Utility function 

'''

import numpy as np

def to_one_hot_enc(arr):

    one_hot_enc = []

    for element in arr[0]:

        sample = [0, 0, 0, 0]

        sample[int(arr)] = 1 

        one_hot_enc.append(sample)

    return np.array(one_hot_enc)


def from_one_hot_enc(arr):

    lst = []
    
    for element in arr:

        if element[0] > element[1]:
            lst.append(1)
        else:
            lst.append(0)
    
    return np.array(lst)


def reshape_images(arr):

    img_list = []

    for img in arr:
        img = np.reshape(np.array(img),(50,50))
        img_list.append(img)

    return np.array(img_list)


def samples_in_class_count(arr):

    samples_in_class = np.bincount(arr.astype(int))

    return samples_in_class[0], samples_in_class[1]

In [38]:
'''
Preparing data 

'''
import numpy as np
from os import getcwd

x_data = np.load(getcwd() + "/training_set/Xtrain_Classification_Part2.npy") # x_data.shape: (7366, 2500)
y_data = np.load(getcwd() + "/training_set/Ytrain_Classification_Part2.npy") # y_data.shape: (7366,)

print(y_data[1])
print(y_data[2])
print(y_data[45])
print(y_data[6])
print(y_data[4])
print(y_data[3])
print(y_data[18])

print(to_one_hot_enc(y_data)[1])
# print(to_one_hot_enc(y_data[0][2]))
# print(to_one_hot_enc(y_data[0][45]))
# print(to_one_hot_enc(y_data[0][6]))
# print(to_one_hot_enc(y_data[0][4]))
# print(to_one_hot_enc(y_data[0][3]))
# print(to_one_hot_enc(y_data[0][18]))



0.0
0.0
0.0
2.0
0.0
2.0
3.0


TypeError: 'numpy.float64' object is not iterable